In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
from datetime import datetime
today = datetime.today().strftime('%Y%m%d')

In [2]:
training_logs_dir = 'evaluation_logs'
watch_dates = ['20200106']#['20191118', '20191218']
contents = [] 
for watch_date in watch_dates:
    contents.append(pd.read_csv('{}.txt'.format(os.path.join(training_logs_dir, watch_date)), sep='\t'))
logs_df = pd.concat(contents, axis=0).reset_index()
logs_df.head()

,index,epoch,seed,train,strategy,accuracy,loss
0,0,1,10,True,freeze,46.922,0.02276536994457245
1,1,1,10,False,freeze,59.35,0.01823648213148117
2,2,2,10,True,freeze,68.006,0.014054816209673882
3,3,2,10,False,freeze,71.57,0.012942347392439843
4,4,3,10,True,freeze,77.188,0.010227634122371673


In [3]:
style_args = {
    'linestyle' : {
        'train': '-',
        'test': '--'
    },
    'colors': {
        'freeze': '#B61616',
        'shuffle': '#5AB616',
        'homogeneous': '#26A4C1',
        'heterogeneous': '#3626C1',
        'max_k_loss': '#ECAA1D',
        'min_k_loss': '#EC2CDB'
    }
}

---

In [4]:
def summarize(df, is_train, strategy, get_acc=True):
    '''
    @df = dataframe with the training/testing logs (epoch, seed, train, shuffle, accuracy, loss)
    @is_train = boolean whether to look at training logs
    @is_shuffle = boolean whether the data is shuffled
    @get_acc = boolean whether to get the accuracy or the loss
    '''
    subset_df = df[['epoch', 'accuracy', 'loss']].loc[(logs_df['train'] == is_train) & (logs_df['strategy'] == strategy)]
    grouped_df = subset_df.groupby('epoch')
    if get_acc:
        return {'mean': grouped_df.mean().values.T[0], 'std': grouped_df.std().values.T[0]}
    else:
        return {'mean': grouped_df.mean().values.T[1], 'std': grouped_df.std().values.T[1]}

In [5]:
def metric_over_epochs(strategies, get_acc=True, data_dir='figures'):
    best_vals = {}
    for is_train in [True, False]:
        plt.close()
        plt.figure(figsize=(12,12))
        for strategy in strategies:
            # data
            metric_dict = summarize(logs_df, is_train, strategy, get_acc=get_acc)
            
            means = metric_dict['mean']
            stds = metric_dict['std']
            upper_shadow = means + stds
            lower_shadow = means - stds
            idxs = range(len(means))
            
            # graph settings
            train_mode = 'train' if is_train else 'test'
            name = '{}/{}'.format(train_mode, strategy)
            best_vals[name] = max(means)
            color = style_args['colors'][strategy]
            linestyle = style_args['linestyle'][train_mode]

            # plotting
            plt.plot(idxs, means, label=name, color=color, linestyle=linestyle)
            plt.fill_between(idxs, means, upper_shadow, color=color, linestyle=linestyle, alpha=0.5)
            plt.fill_between(idxs, lower_shadow, means, color=color, linestyle=linestyle, alpha=0.5)

        plt.title('Change of {} over iterations'.format('accuracy' if get_acc else 'loss'))
        plt.xlabel('Iteration')
        plt.ylabel('{}'.format('Accuracy' if get_acc else 'Loss'))
        plt.legend();
        plt.plot()
        plt.savefig('{}/{}_{}_{}.jpg'.format(data_dir, today, 
                                             'accuracy' if get_acc else 'loss', 
                                             'train' if is_train else 'test'), format='jpg')
        
    print("Best values:")
    print(pd.Series(best_vals))

In [6]:
strategies = list(style_args['colors'].keys())
strategies

['freeze',
 'shuffle',
 'homogeneous',
 'heterogeneous',
 'max_k_loss',
 'min_k_loss']

In [7]:
metric_over_epochs(strategies, get_acc=True)

DataError: No numeric types to aggregate

<Figure size 864x864 with 0 Axes>

In [ ]:
metric_over_epochs(strategies, get_acc=False)